In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

In [3]:
%config Completer.use_jedi = False  # 자동완성 속도 증가

In [14]:
class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        
        w_shape = (input_size, output_size)
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value) # 랜덤한 값으로 초기화된 (input_size, output_size)크기의 행렬 W를 만듬
        
        b_shape = (output_size)
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value) # 0으로 초기화된 (outputsize,) 크기의 벡터 b를 만듬
    
    def __call__(self, inputs): # 정방향 패스를 수행
        return self.activation(tf.matmul(inputs, self.W) + self.b)
    
    @property
    def weights(self): # 가중치를 추출
        return [self.W, self.b]

In [5]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers
        
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

In [6]:
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

In [7]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)
        
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [8]:
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)
    update_weights(gradients, model.weights)
    return average_loss

In [9]:
learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)

In [10]:
from tensorflow.keras import optimizers

optimizer = optimizers.SGD(learning_rate=1e-3)

def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))

In [11]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f'에포크 {epoch_counter}')
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f'{batch_counter}번째 배치 손실: {loss:.2f}')

In [12]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

에포크 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
0번째 배치 손실: 4.66
100번째 배치 손실: 2.21
200번째 배치 손실: 2.21
300번째 배치 손실: 2.08
400번째 배치 손실: 2.18
에포크 1
0번째 배치 손실: 1.89
100번째 배치 손실: 1.85
200번째 배치 손실: 1.83
300번째 배치 손실: 1.70
400번째 배치 손실: 1.80
에포크 2
0번째 배치 손실: 1.57
100번째 배치 손실: 1.55
200번째 배치 손실: 1.50
300번째 배치 손실: 1.42
400번째 배치 손실: 1.50
에포크 3
0번째 배치 손실: 1.31
100번째 배치 손실: 1.32
200번째 배치 손실: 1.24
300번째 배치 손실: 1.20
400번째 배치 손실: 1.27
에포크 4
0번째 배치 손실: 1.12
100번째 배치 손실: 1.14
200번째 배치 손실: 1.05
300번째 배치 손실: 1.04
400번째 배치 손실: 1.11
에포크 5
0번째 배치 손실: 0.97
100번째 배치 손실: 1.00
200번째 배치 손실: 0.91
300번째 배치 손실: 0.92
400번째 배치 손실: 0.99
에포크 6
0번째 배치 손실: 0.87
100번째 배치 손실: 0.90
200번째 배치 손실: 0.81
300번째 배치 손실: 0.83
400번째 배치 손실: 0.91
에포크 7
0번째 배치 손실: 0.79
100번째 배치 손실: 0.82
200번째 배치 손실: 0.73
300번째 배치 손실: 0.77
400번째 배치 손실: 0.84
에포크 8
0번째 배치 손실: 0.72
100번째 배치 손실: 0.75
200번째 배치 손실: 0.67
300번째 배치 손실: 0.71
400번째 배치 손실: 0.79
에포크 9
0번째 배치 손실: 0.67
100번째 배치 손실: 0.70
200번째 배치 손실: 0.62
30

In [13]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f'정확도: {matches.mean():.2f}')

정확도: 0.82


In [34]:
time = tf.Variable(2.)
with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as inner_tape:
        position = 4.9 * time ** 2
    speed = inner_tape.gradient(position, time)
acceleration = outer_tape.gradient(speed, time)
print(speed)
print(acceleration)

tf.Tensor(19.6, shape=(), dtype=float32)
tf.Tensor(9.8, shape=(), dtype=float32)
